# IBM Applied Data Science Capstone

**_Opening a New Hotel in Paris, France_**

- Build a dataframe of arrondissement (neighborhoods) in Paris by web scraping the data from Wikipedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a new Hotel
***
### 1. Importing libraries

In [0]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!pip install geocoder
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium
import folium # map rendering library

print("Libraries imported.")



     |████████████████████████████████| 102kB 2.2MB/s 
Libraries imported.


### 2. Scrap data from Wikipedia page into a DataFrame

In [112]:
# parsing the Table containing Neighborhoods of Paris with very simple Pandas call

table = pd.read_html('https://en.wikipedia.org/wiki/Arrondissements_of_Paris')[2]
table['Name']
    

0                  Louvre
1                  Bourse
2                  Temple
3          Hôtel-de-Ville
4                Panthéon
5              Luxembourg
6          Palais-Bourbon
7                  Élysée
8                   Opéra
9                Entrepôt
10             Popincourt
11                Reuilly
12               Gobelins
13           Observatoire
14              Vaugirard
15                  Passy
16    Batignolles-Monceau
17       Butte-Montmartre
18        Buttes-Chaumont
19           Ménilmontant
Name: Name, dtype: object

In [0]:
# create a new DataFrame with required cleaning to make it easier to work geocoder
kl_df = table
kl_df.drop(kl_df.columns[[0, 2, 3, 4, 5, 6, 7]], axis = 1, inplace = True) 
kl_df = kl_df.rename(columns={'Name': 'Neighborhood'}, inplace=True)

kl_df['Neighborhood'] =  kl_df['Neighborhood'].astype(str) + ',Paris' 
kl_df.head(20)


,Neighborhood
0,"Louvre,Paris"
1,"Bourse,Paris"
2,"Temple,Paris"
3,"Hôtel-de-Ville,Paris"
4,"Panthéon,Paris"
5,"Luxembourg,Paris"
6,"Palais-Bourbon,Paris"
7,"Élysée,Paris"
8,"Opéra,Paris"
9,"Entrepôt,Paris"


In [0]:
# print the number of rows of the dataframe
kl_df.shape

# Total 20 Neighborhoods

(20, 1)

### 3. Get the geographical coordinates

In [0]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Paris, France'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [0]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [0]:
coords

[[48.86102000000005, 2.3358700000000567],
 [48.86910000000006, 2.340220000000045],
 [48.86466000000007, 2.3642600000000584],
 [48.8570946465905, 2.3530753470059596],
 [48.84669992224053, 2.344970061168472],
 [48.84758000000005, 2.340940000000046],
 [48.86079001832194, 2.3185800255352524],
 [48.86931307878868, 2.31687879014301],
 [48.88211997446484, 2.3280599888915554],
 [48.842269945034474, 2.3761199194992475],
 [48.85934002553529, 2.375960043857151],
 [48.84498373420427, 2.389387332244692],
 [48.8344775755156, 2.353492216614434],
 [48.83596997446483, 2.3342200183218913],
 [48.83938001110856, 2.300679930607728],
 [48.85847006939234, 2.284189948929627],
 [48.883500000000026, 2.321900000000028],
 [48.89250000000004, 2.344400000000064],
 [48.87937000000005, 2.3851100000000542],
 [48.86314862455058, 2.387301402267264]]

In [0]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
#df_coords

In [0]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [0]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(20, 3)


,Neighborhood,Latitude,Longitude
0,"Louvre,Paris",48.861020,2.335870
1,"Bourse,Paris",48.869100,2.340220
2,"Temple,Paris",48.864660,2.364260
3,"Hôtel-de-Ville,Paris",48.857095,2.353075
4,"Panthéon,Paris",48.846700,2.344970
5,"Luxembourg,Paris",48.847580,2.340940
6,"Palais-Bourbon,Paris",48.860790,2.318580
7,"Élysée,Paris",48.869313,2.316879
8,"Opéra,Paris",48.882120,2.328060
9,"Entrepôt,Paris",48.842270,2.376120


In [0]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

### 4. Create a map of Paris with neighborhoods superimposed on top

In [0]:
# get the coordinates of Kuala Lumpur
address = 'Paris, France'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris, France {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Paris, France 48.8566969, 2.3514616.


In [0]:
# create map of Paris using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [0]:
# save the map as HTML file
map_kl.save('map_kl.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [0]:
# define Foursquare Credentials and Version 
CLIENT_ID =  # your Foursquare ID
CLIENT_SECRET = # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

**Now, let's get the top 100 venues that are within a radius of 2000 meters.**

In [0]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [0]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2000, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"Louvre,Paris",48.86102,2.33587,Cour Carrée du Louvre,48.860360,2.338543,Pedestrian Plaza
1,"Louvre,Paris",48.86102,2.33587,Musée du Louvre,48.860847,2.336440,Art Museum
2,"Louvre,Paris",48.86102,2.33587,La Vénus de Milo (Vénus de Milo),48.859943,2.337234,Exhibit
3,"Louvre,Paris",48.86102,2.33587,Vestige de la Forteresse du Louvre,48.861577,2.333508,Historic Site
4,"Louvre,Paris",48.86102,2.33587,Palais Royal,48.863236,2.337127,Historic Site


**Let's check how many venues were returned for each neighorhood**

In [0]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"Batignolles-Monceau,Paris",100,100,100,100,100,100
"Bourse,Paris",100,100,100,100,100,100
"Butte-Montmartre,Paris",100,100,100,100,100,100
"Buttes-Chaumont,Paris",100,100,100,100,100,100
"Entrepôt,Paris",100,100,100,100,100,100
"Gobelins,Paris",100,100,100,100,100,100
"Hôtel-de-Ville,Paris",100,100,100,100,100,100
"Louvre,Paris",100,100,100,100,100,100
"Luxembourg,Paris",100,100,100,100,100,100


**Let's find out how many unique categories can be curated from all the returned venues**

In [0]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 178 uniques categories.


In [0]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Pedestrian Plaza', 'Art Museum', 'Exhibit', 'Historic Site',
       'Theater', 'Plaza', 'Bridge', 'Italian Restaurant', 'Garden',
       'Cosmetics Shop', 'Church', 'Hotel', 'French Restaurant', 'Museum',
       'Coffee Shop', 'Fountain', 'Art Gallery', 'Udon Restaurant',
       'Wine Bar', 'Cocktail Bar', 'Bookstore', 'Bakery', 'Restaurant',
       'Pastry Shop', 'Spa', 'Sandwich Place', 'Ice Cream Shop',
       'Breton Restaurant', "Men's Store", 'Clothing Store', 'Tea Room',
       'Seafood Restaurant', 'Lebanese Restaurant', 'Japanese Restaurant',
       'Creperie', 'Bistro', 'Boutique', 'Beer Bar', 'Electronics Store',
       'Concert Hall', 'Cheese Shop', 'Pub', 'Miscellaneous Shop',
       'Souvenir Shop', 'Park', 'Pharmacy', 'Opera House', 'Burger Joint',
       'Tailor Shop', 'Indie Movie Theater'], dtype=object)

In [0]:
# check if the results contain "Hotel"
"Hotel" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each Neighborhood

In [0]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(2000, 179)


,Neighborhoods,African Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auvergne Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Canal,Caribbean Restaurant,Cemetery,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Drive-in Theater,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Fountain,French Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym / Fitness Center,Health Food Store,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Island,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jiangxi Restaurant,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Library,Liquor Store,Lounge,Lyonese Bouchon,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Multiplex,Museum,Music Venue,Nightclub,Opera House,Organic Grocery,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pharmacy,Photography Lab,Pizza Place,Planetarium,Plaza,Pool,Pop-Up Shop,Portuguese Restaurant,Provençal Restaurant,Pub,Radio Station,Record Shop,Recording Studio,Restaurant,Roof Deck,Salad Place,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Soccer Field,Southern / Soul Food Restaurant,Southwestern French Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Park Ride / Attraction,Trail,Trattoria/Osteria,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop
0,"Louvre,Paris",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Louvre,Paris",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Louvre,Paris",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Louvre,Paris",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [0]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(20, 179)


,Neighborhoods,African Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auvergne Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Canal,Caribbean Restaurant,Cemetery,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Drive-in Theater,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Fountain,French Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym / Fitness Center,Health Food Store,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Island,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jiangxi Restaurant,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Library,Liquor Store,Lounge,Lyonese Bouchon,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Multiplex,Museum,Music Venue,Nightclub,Opera House,Organic Grocery,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pharmacy,Photography Lab,Pizza Place,Planetarium,Plaza,Pool,Pop-Up Shop,Portuguese Restaurant,Provençal Restaurant,Pub,Radio Station,Record Shop,Recording Studio,Restaurant,Roof Deck,Salad Place,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Soccer Field,Southern / Soul Food Restaurant,Southwestern French Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Park Ride / Attraction,Trail,Trattoria/Osteria,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop
0,"Batignolles-Monceau,Paris",0.00,0.00,0.00,0.01,0.02,0.00,0.00,0.00,0.01,0.01,0.02,0.04,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.05,0.01,0.01,0.00,0.01,0.00,0.00,0.05,0.02,0.00,0.01,0.01,0.02,0.00,0.02,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.01,0.01,0.00,0.01,0.00,0.00,0.02,0.02,0.01,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.01,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.02,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.05,0.00
1,"Bourse,Paris",0.00,0.00,0.01,0.01,0.05,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.02,0.00,0.06,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.01,0.01,0.03,0.01,0.00,0.00,0.02,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.02,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.03,0.00,0.00,0.02,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.00

In [0]:
len(kl_grouped[kl_grouped["Hotel"] > 0])

19

**Create a new DataFrame for Hotel data only**

In [0]:
kl_mall = kl_grouped[["Neighborhoods","Hotel"]]

In [0]:
kl_mall.head()

,Neighborhoods,Hotel
0,"Batignolles-Monceau,Paris",0.05
1,"Bourse,Paris",0.07
2,"Butte-Montmartre,Paris",0.04
3,"Buttes-Chaumont,Paris",0.00
4,"Entrepôt,Paris",0.03


### 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Paris into 3 clusters.

In [0]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 1, 1, 1, 1, 0, 1, 1], dtype=int32)

In [0]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [0]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Hotel,Cluster Labels
0,"Batignolles-Monceau,Paris",0.05,0
1,"Bourse,Paris",0.07,0
2,"Butte-Montmartre,Paris",0.04,1
3,"Buttes-Chaumont,Paris",0.00,1
4,"Entrepôt,Paris",0.03,1


In [0]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(20, 5)


,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
0,"Batignolles-Monceau,Paris",0.05,0,48.88350,2.32190
1,"Bourse,Paris",0.07,0,48.86910,2.34022
2,"Butte-Montmartre,Paris",0.04,1,48.89250,2.34440
3,"Buttes-Chaumont,Paris",0.00,1,48.87937,2.38511
4,"Entrepôt,Paris",0.03,1,48.84227,2.37612


In [0]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(20, 5)


,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
0,"Batignolles-Monceau,Paris",0.05,0,48.883500,2.321900
1,"Bourse,Paris",0.07,0,48.869100,2.340220
14,"Passy,Paris",0.09,0,48.858470,2.284190
7,"Louvre,Paris",0.07,0,48.861020,2.335870
18,"Vaugirard,Paris",0.06,0,48.839380,2.300680
10,"Observatoire,Paris",0.05,0,48.835970,2.334220
8,"Luxembourg,Paris",0.04,1,48.847580,2.340940
2,"Butte-Montmartre,Paris",0.04,1,48.892500,2.344400
3,"Buttes-Chaumont,Paris",0.00,1,48.879370,2.385110
4,"Entrepôt,Paris",0.03,1,48.842270,2.376120


**Finally, let's visualize the resulting clusters**

In [0]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-3],
        fill=True,
        fill_color=rainbow[cluster-3],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [0]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

#### Cluster 0

In [0]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
0,"Batignolles-Monceau,Paris",0.05,0,48.88350,2.32190
1,"Bourse,Paris",0.07,0,48.86910,2.34022
14,"Passy,Paris",0.09,0,48.85847,2.28419
7,"Louvre,Paris",0.07,0,48.86102,2.33587
18,"Vaugirard,Paris",0.06,0,48.83938,2.30068
10,"Observatoire,Paris",0.05,0,48.83597,2.33422


#### Cluster 1

In [0]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
17,"Temple,Paris",0.03,1,48.864660,2.364260
16,"Reuilly,Paris",0.02,1,48.844984,2.389387
15,"Popincourt,Paris",0.01,1,48.859340,2.375960
13,"Panthéon,Paris",0.03,1,48.846700,2.344970
11,"Opéra,Paris",0.04,1,48.882120,2.328060
9,"Ménilmontant,Paris",0.01,1,48.863149,2.387301
6,"Hôtel-de-Ville,Paris",0.03,1,48.857095,2.353075
5,"Gobelins,Paris",0.04,1,48.834478,2.353492
4,"Entrepôt,Paris",0.03,1,48.842270,2.376120
3,"Buttes-Chaumont,Paris",0.00,1,48.879370,2.385110


#### Cluster 2

In [0]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
12,"Palais-Bourbon,Paris",0.12,2,48.860790,2.318580
19,"Élysée,Paris",0.15,2,48.869313,2.316879


#### Observations:
Most of the Hotels are concentrated aroung the central area of Paris city, with the highest number in cluster 1 and moderate number in cluster 0. On the other hand, cluster 2 has very low number to totally no Hotel in the neighborhoods. This represents a great opportunity and high potential areas to open new Hotel as there is very little to no competition from existing Hotels. 
Meanwhile, Hotel in cluster 1 are likely suffering from intense competition due to oversupply and high concentration of Hotels. From another perspective, this also shows that the oversupply of Hotels mostly happened in the central area of the city, with the suburb area still have very few Hotels. Therefore, this project recommends property developers to capitalize on these findings to open new Hotel in neighborhoods in cluster 2 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new Hotel in neighborhoods in cluster 0 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 1 which already have high concentration of Hotels and suffering from intense competition.